Import packages

In [1]:
!pip install spacy
!pip install spacy-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 40.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.2
    Uninstalling transformers-4.46.2:
      Successfully uninstalled transformers-4.46.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.2.1 requires transformers<5.0.0,>=4.41.0, but you ha

In [2]:
import json
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import spacy_transformers

import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [3]:
# import bibel data
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/NLP/NER/t_bbe.csv')
data.head()

,id,b,c,v,t
0,1001001,1,1,1,At the first God made the heaven and the earth.
1,1001002,1,1,2,And the earth was waste and without form; and ...
2,1001003,1,1,3,"And God said, Let there be light: and there wa..."
3,1001004,1,1,4,"And God, looking on the light, saw that it was..."
4,1001005,1,1,5,"Naming the light, Day, and the dark, Night. An..."


**Column Explanation**

*   **id** : unique id for each verse
*   **b** : book labeled by number
*   **c** : chapter number from the book
*   **v** : verse number of the chapter
*   **t** : text of the verse

Since our work is on Book os Genesis, we select only the book labeled by number one (b == 1)

As annotation takes a long time and needs a team to do it. I just used 10 chapter for annotation and finetuning the spacy pretrained model.

In [4]:
genesis = data[ (data['b'] == 1 & ( data['c'] < 11 ))]

In [5]:
genesis.tail()

,id,b,c,v,t
262,1010028,1,10,28,And Obal and Abimael and Sheba
263,1010029,1,10,29,And Ophir and Havilah and Jobab; all these wer...
264,1010030,1,10,30,"And their country was from Mesha, in the direc..."
265,1010031,1,10,31,"These, with their families and their languages..."
266,1010032,1,10,32,"These are the families of the sons of Noah, in..."


In [6]:
senetences = genesis['t'].tolist()
senetences[0:5]

['At the first God made the heaven and the earth.',
 'And the earth was waste and without form; and it was dark on the face of the deep: and the Spirit of God was moving on the face of the waters.',
 'And God said, Let there be light: and there was light.',
 'And God, looking on the light, saw that it was good: and God made a division between the light and the dark,',
 'Naming the light, Day, and the dark, Night. And there was evening and there was morning, the first day.']

## **File Selection**

Let us save each chapter by their chapter number. This help us to annotate each chapter individually .

In [9]:
for i in tqdm(range(1,51)):

  my_file = data[ data['c'] == i ] # chapter select

  sentences = []
  for sent in my_file['t']:
    sentences.append(sent)
  sentences = ' '.join(sentences)  # convert to string

  file_path = '/content/drive/MyDrive/NLP/NER/GENESIS' + '/bibel_' + str(i) + '.txt' # create path name to save


  with open(file_path, 'w') as file:
    file.write(sentences)

100%|██████████| 50/50 [00:00<00:00, 122.26it/s]


***Now go to doccano annotation work and when finished come back.***

## **Doccano to Spacy Conversion**

Spacy need data oriented by its own format. So we need to convert the doccano formated annotation to spacy.

### Example of doccano format

```
{ "id":1,
  "text": "In the beginning, God created the heavens and the earth.",
  "entities": [
    {"id":1, "start_offset": 3, "end_offset": 16, "label": "TIME"},
    {"id":2, "start_offset": 18, "end_offset": 21, "label": "PERSON"},
    {"id":3, "start_offset": 31, "end_offset": 38, "label": "LOCATION"},
    {"id":4, "start_offset": 43, "end_offset": 48, "label": "LOCATION"}
  ]
}


```

### Expected spacy format

```
[
    (
        "In the beginning, God created the heavens and the earth.",
            [
                (3, 16, "TIME"),
                (18, 21, "PERSON"),
                (31, 38, "LOCATION"),
                (43, 48, "LOCATION")
            ]
    )
]



This is a function used to convert doccano formated file into spacy oriented format.

In [10]:
def direct_convert(json_data):
  text = json_data['text']
  annot = json_data['entities']
  annot
  my_annot = []
  for entity in annot:

    start = entity['start_offset']
    end = entity['end_offset']
    label = entity['label']


    my_annot.append((start, end, label))
  finall = (text, my_annot)

  return finall

First save your annotated files into one folder, advice able  to used google drive. Then by using loop iterate to each annotated file. For each json file call the function to convert.

In [11]:
spacy_data = []
i = 0
for i in range(1,11):
  file_path = '/content/drive/MyDrive/NLP/NER/FROM DOCCANO/bibel_' + str(i) + '.jsonl'
  data = json.load(open(file_path))

  spacy_data.append( direct_convert(data) ) # merge all files into one variable

In [12]:
spacy_data[0]

('At the first God made the heaven and the earth. And the earth was waste and without form; and it was dark on the face of the deep: and the Spirit of God was moving on the face of the waters. And God said, Let there be light: and there was light. And God, looking on the light, saw that it was good: and God made a division between the light and the dark, Naming the light, Day, and the dark, Night. And there was evening and there was morning, the first day. And God said, Let there be a solid arch stretching over the waters, parting the waters from the waters. And God made the arch for a division between the waters which were under the arch and those which were over it: and it was so. And God gave the arch the name of Heaven. And there was evening and there was morning, the second day. And God said, Let the waters under the heaven come together in one place, and let the dry land be seen: and it was so. And God gave the dry land the name of Earth; and the waters together in their place we

## **Split Data**

The training process needs train and test data. So split the spacy formated data.

In [13]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(spacy_data, test_size=0.2) # 20% for test data

In [14]:
len(train), len(test)

(8, 2)

## **Convert to DocBin**

DocBin handle annotation to be ready for training

In [15]:
# Load a pre-trained spaCy model (e.g., 'en_core_web_sm')
nlp = spacy.load("en_core_web_sm")

In [16]:
def to_docbin(data, path):
  db = DocBin()
  for text, annotations in data:
      doc = nlp(text)
      ents = []
      for start, end, label in annotations:
          span = doc.char_span(start, end, label=label)
          if span:
            ents.append(span)
          else:
            print("Skipping entity:", (start, end, label))
      try:
        doc.ents = ents
        db.add(doc)
      except:
        pass
  db.to_disk(path)

In [17]:
to_docbin(train, "./train.spacy")
to_docbin(test, "./test.spacy")

Skipping entity: (1705, 1716, 'QUANTITY')
Skipping entity: (2962, 2990, 'DATE')
Skipping entity: (2354, 2361, 'PERSON')
Skipping entity: (189, 192, 'PERSON')
Skipping entity: (902, 905, 'PERSON')
Skipping entity: (22, 28, 'LOC')


## **Configuring Spacy**

we need to create a configuration file from base file. Check the following link for [Spacy Usage](https://spacy.io/usage/training)

In [18]:
# change the path based on your location
!python -m spacy init fill-config '/content/drive/MyDrive/NLP/NER/config_files/base_config.cfg' config.cfg

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


## **Debug Spacy data**

We should check our spacy data. open config file and give path of train and test data. This help us to see if overlapped annotation is there, low data labels, invalid annotation and more.

In [19]:
!python -m spacy debug data /content/config.cfg --paths.train /content/train.spacy --paths.dev /content/test.spacy

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(

============================ Data file validation ============================
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
tokenizer_config.json: 100% 25.0/25.0 [00:00<00:00, 183kB/s]
config.json: 100% 481/481 [00:00<00:00, 3.25MB/s]
vocab.json: 100% 899k/89

*    We have 4 warning and 1 error.
*    The error is raised due to low number of example and 4 warnings due to small example of labels.

## **Train Spacy pretrained Model**

In [20]:
!python -m spacy train /content/config.cfg --output ./output --paths.train /content/train.spacy --paths.dev /content/test.spacy --gpu-id 0

✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transform

We can abort the training process if we won't to make further changes to the performace.

## **Model Evaluation**

In [21]:
# Load the trained spaCy model
nlp = spacy.load('./output/model-best')

In [23]:
text = "Noah built an ark, and God made a covenant with him at Mount Ararat. Euphritus"

# Process the text with the loaded model
doc = nlp(text)

# Print the entities
print("Entities:", [(ent.text, ent.label_) for ent in doc.ents])

Entities: [('Noah', 'PERSON'), ('God', 'Creator'), ('Mount Ararat', 'PERSON'), ('Euphritus', 'PERSON')]


The result shows that selection of entities, one thing to analyze is '**Euphritus** is not a **PERSON**, rather a river. This raised because the model has insufficient amount labels referes to river. On the other hand the book of genesis has many '**PERSON**' which is going to bias the models to predict as a '**PERSON**' most of the texts.

## **Recommendation**

The more annotation file, the more generalization performance. I recommend to use 50 to 200 number of examples for the finetuning process.

## **Save packages**

In [ ]:
!pip freeze > requirements.txt

In [ ]:
from google.colab import files

files.download('requirements.txt')


# **MANY REGARDS ...**